## Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import repo

In [ ]:
%rm -rf /content/learning
import getpass
!git clone --branch seq2seq https://{getpass.getpass()}@github.com/JoaoJanini/learning

··········
Cloning into 'learning'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 223 (delta 108), reused 170 (delta 67), pack-reused 0
Receiving objects: 100% (223/223), 241.15 KiB | 6.35 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [ ]:
!sleep 5

In [ ]:
%cd learning

/content/learning


In [ ]:
%mkdir data

In [ ]:
!pip install -r requirements.txt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Sequence to Sequence

In [ ]:
import sys
path = "./seq2seq"
if path not in sys.path:
  sys.path.append(path)
else:
  print(path + "already in path")

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 './seq2seq']

In [ ]:
model_path = "/content/drive/MyDrive/Coding/trained_models_sequence_2_label/base_model"

## Training

In [ ]:
"""
@author : Hyunwoong
@when : 2019-10-22
@homepage : https://github.com/gusdnd852
"""
import torch

# GPU device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# dataset settings
SEQUENCE_LEN = 10
TRAINING_RATIO = 0.8
WIRELINE_LOGS_HEADER = ["GR", "NPHI"]
LABEL_COLUMN_HEADER = ["FORCE_2020_LITHOFACIES_LITHOLOGY"]

# model parameter setting
batch_size = 640
max_len = 256
d_model = 512
n_layers = 6
n_heads = 8
ffn_hidden = 2048
drop_prob = 0.1

# optimizer parameter setting
init_lr = 1e-4
factor = 0.9
adam_eps = 5e-9
patience = 10
warmup = 100
epoch = 10
clip = 1.0
weight_decay = 5e-4
inf = float('inf')

correct_on_train = []
correct_on_test = []

In [ ]:
"""
@author : Hyunwoong
@when : 2019-10-29
@homepage : https://github.com/gusdnd852
"""
from torch.utils.data import DataLoader
from seq2seq.dataset.dataset import WellsDataset
from torch.utils.data import random_split

train_dataset = WellsDataset(dataset_type = "train",sequence_len = SEQUENCE_LEN, model_type="seq2seq",feature_columns = WIRELINE_LOGS_HEADER, label_columns = LABEL_COLUMN_HEADER)
test_dataset = WellsDataset(dataset_type = "test", sequence_len = SEQUENCE_LEN, model_type="seq2seq",feature_columns = WIRELINE_LOGS_HEADER, label_columns = LABEL_COLUMN_HEADER, scaler = train_dataset.scaler, output_len= train_dataset.output_len)

DATA_LEN = train_dataset.train_len 
d_input = train_dataset.input_len  
d_channel = train_dataset.channel_len  
d_output = train_dataset.output_len
TRAIN_DATA_LEN = int(DATA_LEN * TRAINING_RATIO)
train_dataset, validation_dataset = random_split(train_dataset, lengths=[TRAIN_DATA_LEN, DATA_LEN - TRAIN_DATA_LEN])

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=True)


print('data structure: [lines, timesteps, features]')
print(f'train data size: [{DATA_LEN, d_input, d_channel}]')
print(f'Number of classes: {d_output}')

dec_voc_size = d_output




data structure: [lines, timesteps, features]
train data size: [(76492, 10, 2)]
Number of classes: 12


### train_functions

In [ ]:
"""
@author : Hyunwoong
@when : 2019-10-22
@homepage : https://github.com/gusdnd852
"""
import math
import time
from torch import nn, optim
from torch.optim import Adam
import torch
from seq2seq.transformer.transformer import Transformer
from seq2seq.util.epoch_timer import epoch_time

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.kaiming_uniform(m.weight.data)

model = Transformer(d_model=d_model,
                    d_channel=d_channel,
                    d_input=d_input,
                    dec_voc_size=dec_voc_size,
                    max_len=max_len,
                    ffn_hidden=ffn_hidden,
                    n_head=n_heads,
                    n_layers=n_layers,
                    drop_prob=drop_prob,
                    device=device
                    ).to(device)
print(f'The model has {count_parameters(model):,} trainable parameters')
model.apply(initialize_weights)
optimizer = Adam(params=model.parameters(),
                 lr=init_lr,
                 weight_decay=weight_decay,
                 eps=adam_eps)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 verbose=True,
                                                 factor=factor,
                                                 patience=patience)

criterion = nn.CrossEntropyLoss()

def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, (src, trg) in enumerate(iterator):
        optimizer.zero_grad()
        src = src.to(device)
        trg = trg.to(device)
        # Add <sos> token to the beginning of the target sentence
        trg = torch.cat(((torch.zeros((trg.shape[0], 1)).to(device)), trg), dim=1).long()
        output = model(src, trg[:, :-1])
        
        output_reshape = output.contiguous().view(-1, output.shape[-1])
        trg = trg[:, 1:].contiguous().view(-1)

        loss = criterion(output_reshape, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()
        print("step :", round((i / len(iterator)) * 100, 2), "% , loss :", loss.item())

    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    batch_bleu = []

    with torch.no_grad():
        accuracy = torchmetrics.Accuracy().to(device)
        for i, (src, trg) in enumerate(iterator):
            src = src.to(device)
            trg = trg.to(device)
            trg = torch.cat(((torch.zeros((trg.shape[0], 1)).to(device)), trg), dim=1).long()
            output = model(src, trg[:, :-1])
            output_reshape = output.contiguous().view(-1, output.shape[-1])
            trg_true = trg[:, 1:].contiguous().view(-1)
            loss = criterion(output_reshape, trg_true)
            epoch_loss += loss.item()
            _, label_index = torch.max(output_reshape.data, dim=-1)
            acc = accuracy(label_index, trg_true)
        acc = accuracy.compute()

        accuracy.reset()
    return epoch_loss / len(iterator), acc

def run(total_epoch, best_loss):
    train_losses, test_losses, bleus = [], [], []
    for step in range(total_epoch):
        start_time = time.time()
        train_loss = train(model, train_loader, optimizer, criterion, clip)
        test_loss, test_accuracy = evaluate(model, test_loader, criterion)
        valid_loss, validation_accuracy = evaluate(model, validation_loader, criterion)
        end_time = time.time()

        if step > warmup:
            scheduler.step(valid_loss)

        train_losses.append(train_loss)
        test_losses.append(valid_loss)
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), model_path+'-{0}.pt'.format(valid_loss))

        f = open(model_path+'-train_loss.txt', 'w')
        f.write(str(train_losses))
        f.close()

        f = open(model_path+'-bleu.txt', 'w')
        f.write(str(bleus))
        f.close()

        f = open(model_path+'-test_loss.txt', 'w')
        f.write(str(test_losses))
        f.close()

        print(f'Epoch: {step + 1} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\tVal Loss: {valid_loss:.3f} |  Val PPL: {math.exp(valid_loss):7.3f}')
        print(f"Val Accuracy : {validation_accuracy}")
        print(f'\tTest Loss: {test_loss:.3f} |  Val PPL: {math.exp(test_loss):7.3f}')
        print(f" Test Accuracy : {test_accuracy}")
        print(f'\tBLEU Score: {valid_loss:.3f}')

The model has 52,091,662 trainable parameters


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.


### run

In [ ]:
run(total_epoch=epoch, best_loss=inf)

step : 0.0 % , loss : 3.7512083053588867
step : 1.04 % , loss : 1.7347835302352905
step : 2.08 % , loss : 2.5026466846466064
step : 3.12 % , loss : 1.5871424674987793
step : 4.17 % , loss : 1.5979477167129517
step : 5.21 % , loss : 1.3139911890029907
step : 6.25 % , loss : 1.4188411235809326
step : 7.29 % , loss : 1.2371487617492676
step : 8.33 % , loss : 1.2896125316619873
step : 9.38 % , loss : 1.3630881309509277
step : 10.42 % , loss : 1.197631597518921
step : 11.46 % , loss : 1.2907967567443848
step : 12.5 % , loss : 1.2295007705688477
step : 13.54 % , loss : 1.232176423072815
step : 14.58 % , loss : 1.2562744617462158
step : 15.62 % , loss : 1.1833546161651611
step : 16.67 % , loss : 1.1943281888961792
step : 17.71 % , loss : 1.156787395477295
step : 18.75 % , loss : 1.2389497756958008
step : 19.79 % , loss : 1.2077631950378418
step : 20.83 % , loss : 1.1845499277114868
step : 21.88 % , loss : 1.1469180583953857
step : 22.92 % , loss : 1.165919303894043
step : 23.96 % , loss : 1.1

KeyboardInterrupt: ignored

## Training